Первоначальная загрузка модулей может занимать много времени: с яндекс.диска будут скачиваться модели суммарно 1 гб.

In [1]:
from merito.sphere_classifier import get_predict_for_resume
from merito.vacancy_predictor import predict_salary_by_vacancy
from merito.job_title_correction import text_correction
import pandas as pd

/Users/bondarchukgleb/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Models are already downloaded!
Models are already downloaded!
Models are already downloaded!


### Пример работы модели автокоррекции написания профессий и алгоритма их преобразования

Функция **text_correction** принимает на вход текст с написанной профессией и возвращает его корректное исправленное описание и группу и подгруппу, к которой оно относится (т.е. преобразованные и кластеризированные).

In [9]:
# примеры

job = ["дврник", "дизайнер", "старший геолог", "геолог нефтяник", "свинопас", "экномист"]
for j in job:
    res = text_correction(j)
    print("Исходное название:", j, "| Отредактированное:", res[0], "| Сфера:", res[1][0], "| Подсфера:", res[1][1])

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 3701.95it/s]


Исходное название: дврник | Отредактированное: дворник | Сфера: Рабочий персонал | Подсфера: дворник


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2882.68it/s]


Исходное название: дизайнер | Отредактированное: дизайнер | Сфера: Искусство, развлечения, масс-медиа | Подсфера: дизайнер


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4739.33it/s]


...

Исходное название: геолог нефтяник | Отредактированное: геолог нефтяник | Сфера: Добыча сырья | Подсфера: геолог


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4048.56it/s]


Исходное название: свинопас | Отредактированное: свинопас | Сфера: Искусство, развлечения, масс-медиа | Подсфера: свинопас


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4293.04it/s]

Исходное название: экномист | Отредактированное: экономист | Сфера: Бухгалтерия, управленческий учет, финансы предприятия | Подсфера: экономист


## Определение сфер и зарплатных ожиданий

Определение сфер и зарплатных ожиданий упаковано в отдельные аккуратные модули, которые можно применить всего в одну строчку кода.

In [3]:
test_res = pd.read_csv("test_data/TEST_RES.csv")
test_sal = pd.read_csv("test_data/TEST_SAL.csv")

In [4]:
test_res["job_title"] = get_predict_for_resume(test_res)

100%|███████████████████████████████████| 1956/1956 [00:00<00:00, 102323.06it/s]


In [5]:
test_sal["salary"] = predict_salary_by_vacancy(test_sal)

100%|██████████████████████████████████████| 1956/1956 [00:10<00:00, 194.95it/s]


In [8]:
ids = pd.concat([test_res["id"], test_sal["id"]], axis=0)
job_title = pd.concat([test_res["job_title"], pd.Series([None] * len(test_sal))], axis=0)
salary = pd.concat([pd.Series([None] * len(test_res)), pd.Series(test_sal["salary"])], axis=0)
task_type = ["RES"] * len(test_res) + ["SAL"] * len(test_sal)

In [9]:
sub = pd.DataFrame({"id": ids, "job_title": job_title, "task_type": task_type, "salary": salary})

In [10]:
sub.to_csv("SUBMISSION.csv", index=False)

In [16]:
import merito

In [22]:
merito.vacancy_predictor.reg.get_feature_importance(prettified=True).head(10)

,Feature Id,Importances
0,nearest_mean_sal,21.938544
1,schedule_type,10.975354
2,code_profession,8.802224
3,code_professional_sphere,4.496476
4,vacancy_address_longitude,4.073716
5,vacancy_address_latitude,3.918239
6,required_experience,3.885977
7,source_type,2.953826
8,company_business_size,2.470253
9,mean_regionName_salary,2.244365
